# dev-tooling.ipynb

Develop the API code to support tooling

## LiteLLM Original Example

In [ ]:
import litellm
import importlib
print(importlib.metadata.version("litellm"))
import json
# set openai api key
import os
os.environ['OPENAI_API_KEY'] = os.environ['MY_OPENAI_API_KEY']
MODEL = "openai/gpt-4o"

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


def test_parallel_function_call():
    try:
        # Step 1: send the conversation and available functions to the model
        messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
        tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_current_weather",
                    "description": "Get the current weather in a given location",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                        },
                        "required": ["location"],
                    },
                },
            }
        ]
        response = litellm.completion(
            model=MODEL,
            messages=messages,
            tools=tools,
            tool_choice="auto",  # auto is default, but we'll be explicit
        )
        print("\nFirst LLM Response:\n", response)
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls

        print("\nLength of tool calls", len(tool_calls))

        # Step 2: check if the model wanted to call a function
        if tool_calls:
            # Step 3: call the function
            # Note: the JSON response may not always be valid; be sure to handle errors
            available_functions = {
                "get_current_weather": get_current_weather,
            }  # only one function in this example, but you can have multiple
            messages.append(response_message)  # extend conversation with assistant's reply

            # Step 4: send the info for each function call and function response to the model
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                )
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )  # extend conversation with function response
            second_response = litellm.completion(
                model=MODEL,
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print("\nSecond LLM response:\n", second_response)
            return second_response
    except Exception as e:
      print(f"Error occurred: {e}")

test_ret1 = test_parallel_function_call()
test_ret1

1.66.0

First LLM Response:
 ModelResponse(id='chatcmpl-BPYZS5X7TqoZBFq0eChv0MZ7IJcZ5', created=1745430790, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_a6889ffe71', choices=[Choices(finish_reason='tool_calls', index=0, message=Message(content=None, role='assistant', tool_calls=[ChatCompletionMessageToolCall(function=Function(arguments='{"location": "San Francisco, CA"}', name='get_current_weather'), id='call_CM8YvUssPu0esXDocqERKxoV', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Tokyo, Japan"}', name='get_current_weather'), id='call_ObiKjcD4OkO4hapX5JFtkWom', type='function'), ChatCompletionMessageToolCall(function=Function(arguments='{"location": "Paris, France"}', name='get_current_weather'), id='call_70kJUfm5HTM0S1z9S2oIuheG', type='function')], function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=69, prompt_tokens=85, total_tokens=154, completion

/home/johann/software/anaconda/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...oV', 'type': 'function'}, input_type=dict])
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...om', 'type': 'function'}, input_type=dict])
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...eG', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(



Second LLM response:
 ModelResponse(id='chatcmpl-BPYZTITWPO9sSPdsbCHlITLOUp7q4', created=1745430791, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_90122d973c', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Here's the current weather in the following cities:\n\n- **San Francisco, CA**: The temperature is 72°F.\n- **Tokyo, Japan**: The temperature is 10°C.\n- **Paris, France**: The temperature is 22°C.", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=53, prompt_tokens=158, total_tokens=211, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')


ModelResponse(id='chatcmpl-BPYZTITWPO9sSPdsbCHlITLOUp7q4', created=1745430791, model='gpt-4o-2024-08-06', object='chat.completion', system_fingerprint='fp_90122d973c', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Here's the current weather in the following cities:\n\n- **San Francisco, CA**: The temperature is 72°F.\n- **Tokyo, Japan**: The temperature is 10°C.\n- **Paris, France**: The temperature is 22°C.", role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]))], usage=Usage(completion_tokens=53, prompt_tokens=158, total_tokens=211, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0, text_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=0, cached_tokens=0, text_tokens=None, image_tokens=None)), service_tier='default')

## Own Lib

In [1]:
import os, sys
from typing import Optional, List, Dict
import docstring_parser
sys.path.append(os.path.join(".."))
from llms_wrapper.llms import LLMS, toolnames2funcs, get_func_by_name
from llms_wrapper.config import update_llm_config

In [2]:
config = dict(
    llms=[
        # OpenAI
        # https://platform.openai.com/docs/models
        dict(llm="openai/gpt-4o"),
        dict(llm="openai/gpt-4o-mini"),
        # dict(llm="openai/o1"),        # restricted
        # dict(llm="openai/o1-mini"),   # restricted
        # Google Gemini
        # https://ai.google.dev/gemini-api/docs/models/gemini
        dict(llm="gemini/gemini-2.0-flash-exp"),
        dict(llm="gemini/gemini-1.5-flash"),
        dict(llm="gemini/gemini-1.5-pro"),
        # Anthropic
        # https://docs.anthropic.com/en/docs/about-claude/models
        dict(llm="anthropic/claude-3-5-sonnet-20240620"),
        dict(llm="anthropic/claude-3-opus-20240229"),
        # Mistral
        # https://docs.mistral.ai/getting-started/models/models_overview/
        dict(llm="mistral/mistral-large-latest"),
        # XAI
        # dict(llm="xai/grok-2"),     # not mapped by litellm yet?
        dict(llm="xai/grok-beta"),
        # Groq
        # https://console.groq.com/docs/models
        dict(llm="groq/llama3-70b-8192"),
        dict(llm="groq/llama-3.3-70b-versatile"),
        # Deepseek
        # https://api-docs.deepseek.com/quick_start/pricing
        dict(llm="deepseek/deepseek-chat"),
    ],
    providers = dict(
        openai = dict(api_key_env="MY_OPENAI_API_KEY"),
        gemini = dict(api_key_env="MY_GEMINI_API_KEY"),
        anthropic = dict(api_key_env="MY_ANTHROPIC_API_KEY"),
        mistral = dict(api_key_env="MY_MISTRAL_API_KEY"),
        xai = dict(api_key_env="MY_XAI_API_KEY"),    
        groq = dict(api_key_env="MY_GROQ_API_KEY"),
        deepseek = dict(api_key_env="MY_DEEPSEEK_API_KEY"),
    )
)
config = update_llm_config(config)

In [3]:
llms = LLMS(config)

In [4]:
llms.list_aliases()

['openai/gpt-4o',
 'openai/gpt-4o-mini',
 'gemini/gemini-2.0-flash-exp',
 'gemini/gemini-1.5-flash',
 'gemini/gemini-1.5-pro',
 'anthropic/claude-3-5-sonnet-20240620',
 'anthropic/claude-3-opus-20240229',
 'mistral/mistral-large-latest',
 'xai/grok-beta',
 'groq/llama3-70b-8192',
 'groq/llama-3.3-70b-versatile',
 'deepseek/deepseek-chat']

In [5]:
def query_names(where_clause: str) -> List[str]: 
    """
    Query the customer database and return a list of matching names. 

    This function queries the customer database using the conditions in the where clause and returns
    a list of matching customers. The where clause may use the DB fields "city", "company_name",
    "country" and "since_date" to limit the returned customer list. The where clause can also 
    be followed by a limit clause to limit the number of returned names. 

    :param where_clause: the string containing the where and optionally limit clauses in SQL query format
    :type where_clause: string
    :return: a list of matching customer names
    :rtype: array
    """
    return ["Monica Schmidt", "Harald Mueller"]


## Tooling description creation

In [6]:
from typing import Dict, List
def testfunc1(parm1: str, parm2: Dict[str, List[int]], parm3: bool = True, parm4: List = None) -> list[dict[str, str]]:
    """ 
    Short description of the function.

    Here is a longer description of the function. It can be multiple lines long and
    can contain any information that is relevant to the function.
    

    :param parm1: the first parameter
    :type parm1: str
    :param parm2: the second parameter
    :param parm3: the third parameter
    :type parm3: boolean
    :param parm4: the fourth parameter
    :type parm4: {"type": "array!", "items": {"type": "object", "properties": {"name": {"type": "string"}, "city": {"type": "string"}}}}
    :return: A list of person information, each having a name and city and optional other fields
    :rtype: list[dict[str, str]]
    """
    return [{"name": "Monica Schmidt", "city": "Berlin"}, {"name": "Harald Mueller", "city": "Munich"}]

In [7]:
import inspect
argspec1 = inspect.getfullargspec(testfunc1)
print(argspec1)

FullArgSpec(args=['parm1', 'parm2', 'parm3', 'parm4'], varargs=None, varkw=None, defaults=(True, None), kwonlyargs=[], kwonlydefaults=None, annotations={'return': list[dict[str, str]], 'parm1': <class 'str'>, 'parm2': typing.Dict[str, typing.List[int]], 'parm3': <class 'bool'>, 'parm4': typing.List})


In [8]:
for pname in argspec1.args:
    print(pname, argspec1.annotations.get(pname, None))

parm1 <class 'str'>
parm2 typing.Dict[str, typing.List[int]]
parm3 <class 'bool'>
parm4 typing.List


In [9]:
parm2a = argspec1.annotations.get("parm2", None)

In [10]:
parm2a._args__

AttributeError: type object 'dict' has no attribute '_args__'

In [11]:
argspec2 = inspect.signature(testfunc1)
print(argspec2)
for pname, pval in argspec2.parameters.items():
    print(f"{pname}: type={pval.annotation}, default={pval.default}")
print(argspec2.return_annotation)
print(argspec2.return_annotation.__name__)
print(argspec2.return_annotation.__args__)


(parm1: str, parm2: Dict[str, List[int]], parm3: bool = True, parm4: List = None) -> list[dict[str, str]]
parm1: type=<class 'str'>, default=<class 'inspect._empty'>
parm2: type=typing.Dict[str, typing.List[int]], default=<class 'inspect._empty'>
parm3: type=<class 'bool'>, default=True
parm4: type=typing.List, default=None
list[dict[str, str]]
list
(dict[str, str],)


In [12]:
parm2b = argspec2.parameters.get("parm2", None)

In [13]:
parm2b.annotation.__args__

(str, typing.List[int])

In [14]:
# function which recursively inspects a python type annotation and prints either an integral type or recursively a parametrized type and its parameters
def print_annotation(annotation):
    if hasattr(annotation, '__name__'):
        print(annotation.__name__)
    elif hasattr(annotation, '__args__'):
        print("DEBUG: have args: ", annotation.__args__)
        print(annotation.__origin__.__name__, end='(')
        for arg in annotation.__args__:
            print_annotation(arg)
            print(', ', end='')
        print(')')
    else:
        print(annotation)
print_annotation(parm2b)

parm2: Dict[str, List[int]]


In [18]:
doc = docstring_parser.parse(testfunc1.__doc__)
docret = doc.returns

In [105]:
docret.__dict__

{'args': ['return'],
 'description': 'A list of person information, each having a name and city and optional other fields',
 'type_name': 'list[dict[str, str]]',
 'is_generator': False,
 'return_name': None}

In [148]:
# function to create a single parameter description schema from the parameter name, python type and description
import typing
from typing import Optional, List, Dict, Union
import docstring_parser
from typing import get_origin, get_args
import json
import sys

def ptype2schema(py_type):
    # Handle bare None
    if py_type is type(None):
        return {"type": "null"}

    origin = get_origin(py_type)
    args = get_args(py_type)

    if origin is None:
        # Base types
        if py_type is str:
            return {"type": "string"}
        elif py_type is int:
            return {"type": "integer"}
        elif py_type is float:
            return {"type": "number"}
        elif py_type is bool:
            return {"type": "boolean"}
        elif py_type is type(None):
            return {"type": "null"}
        else:
            return {"type": "string"}  # Fallback

    elif origin is list or origin is typing.List:
        item_type = ptype2schema(args[0]) if args else {"type": "string"}
        return {"type": "array", "items": item_type}

    elif origin is dict or origin is typing.Dict:
        key_type, val_type = args if args else (str, str)
        # JSON Schema requires string keys
        if key_type != str:
            raise ValueError("JSON object keys must be strings")
        return {"type": "object", "additionalProperties": ptype2schema(val_type)}

    elif origin is typing.Union:
        # Flatten nested Union
        flat_args = []
        for arg in args:
            if get_origin(arg) is typing.Union:
                flat_args.extend(get_args(arg))
            else:
                flat_args.append(arg)

        schemas = [ptype2schema(a) for a in flat_args]
        return {"anyOf": schemas}

    elif origin is typing.Literal:
        return {"enum": list(args)}

    else:
        return {"type": "string"}  # fallback for unsupported/unknown
    
def function2schema(func, include_return_type=True):
    doc = docstring_parser.parse(func.__doc__)
    desc = doc.short_description + "\n\n" + doc.long_description if doc.long_description else doc.short_description
    if not desc:
        raise ValueError("Function docstring is empty")
    argdescs = {arg.arg_name: arg.description for arg in doc.params}    
    argtypes = {}
    for arg in doc.params:
        argtype = arg.type_name
        print(f"Debug: argtype for {arg.arg_name}: {argtype}, type: {type(argtype)}")
        # if the argtype is not specified, skip, we will use the argument type
        if argtype is None:
            print(f"Debug: argtype for {arg.arg_name} is None, skipping")
            continue
        # if the argtype starts with a brace, we assume it is already specified as a JSON schema
        if argtype.startswith("{"):
            print(f"Debug: argtype for {arg.arg_name} is a JSON schema ({argtype}), using as is")                        
            argtypes[arg.arg_name] = json.loads(argtype)
        else:
            # otherwise, we assume it is a python type            
            argtypes[arg.arg_name] = ptype2schema(argtype)
            print(f"Debug: argtype for {arg.arg_name} is a python type, converted to JSON schema {argtypes[arg.arg_name]}")
    print(f"Debug: argtypes: {argtypes}")
    retdesc = doc.returns.description if doc.returns else ""
    if not retdesc:
        raise ValueError("Function return type is not specified in docstring")
    retschema = ptype2schema(func.__annotations__.get("return", None))
    desc = desc + "\n\n" + "The function returns: " + str(retdesc)
    if include_return_type:
        desc = desc + "\n\n" + "The return type is: " + str(retschema)
    sig = inspect.signature(func)
    parameters = sig.parameters

    props = {}
    required = []

    for name, param in parameters.items():
        if name == 'self':
            continue

        if name in argtypes:
            print(f"Debug: argtype for {name} is in argtypes, using as is: {argtypes[name]}")
            schema = argtypes[name]
        else:
            print(f"Debug: argtype for {name} is not in argtypes, using function signature {name}: {param}")
            # Use the type annotation if available, otherwise default to string
            ptype = param.annotation if param.annotation != inspect.Parameter.empty else str
            schema = ptype2schema(ptype)
        schema["description"] = argdescs.get(name, "")

        if param.default != inspect.Parameter.empty:
            schema["default"] = param.default
        else:
            required.append(name)

        props[name] = schema

    return {
        "name": func.__name__,
        "description": desc,
        "parameters": {
            "type": "object",
            "properties": props,
            "required": required
        }
    }
    


In [149]:
print(parm2b)
ptype2schema(parm2b.annotation)

parm2: Dict[str, List[int]]


{'type': 'object',
 'additionalProperties': {'type': 'array', 'items': {'type': 'integer'}}}

In [19]:
function2schema(testfunc1)

NameError: name 'function2schema' is not defined

In [28]:
tmpfunc("city='Berlin' and company_name='Acme Corp' and since_date='2023-01-01' and limit=10")

['Monica Schmidt', 'Harald Mueller']

In [29]:
tools=llms.make_tooling(query_names)
tools

[{'type': 'function',
  'function': {'name': 'query_names',
   'description': 'Query the customer database and return a list of matching names. \n\nThis function queries the customer database using the conditions in the where clause and returns\na list of matching customers. The where clause may use the DB fields "city", "company_name",\n"country" and "since_date" to limit the returned customer list. The where clause can also \nbe followed by a limit clause to limit the number of returned names.',
   'parameters': {'type': 'object',
    'properties': {'where_clause': {'type': 'string',
      'description': 'the string containing the where and optionally limit clauses in SQL query format'}},
    'required': ['where_clause']}}}]

In [30]:
llms.supports_function_calling("openai/gpt-4o")

True

In [31]:
llms.supports_function_calling("openai/gpt-4o", parallel=True)

False

In [32]:
msgs = llms.make_messages("Give me the names of customers in New York which have been customers since 2023 or longer")
msgs

[{'content': 'Give me the names of customers in New York which have been customers since 2023 or longer',
  'role': 'user'}]

In [33]:
ret = llms.query("openai/gpt-4o", messages=msgs, tools=tools, return_cost=True)
ret["answer"], ret["error"]

/home/johann/software/anaconda/envs/llms_wrapper/lib/python3.11/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `ChatCompletionMessageToolCall` - serialized value may not be as expected [input_value={'function': {'arguments'...xq', 'type': 'function'}, input_type=dict])
  return self.__pydantic_serializer__.to_python(


('The customers in New York who have been customers since 2023 or longer are:\n\n- Monica Schmidt\n- Harald Mueller',
 '')

In [13]:
ret

{'elapsed_time': 1.8553009033203125,
 'cost': 0.0015425,
 'n_completion_tokens': 63,
 'n_prompt_tokens': 365,
 'n_total_tokens': 428,
 'finish_reason': 'stop',
 'answer': 'Here are the names of customers in New York who have been customers since 2023 or longer:\n\n1. Monica Schmidt\n2. Harald Mueller',
 'error': '',
 'ok': True}

## Test final make_tooling function

In [9]:
llms.make_tooling(testfunc1)

[{'name': 'testfunc1',
  'description': "Short description of the function.\n\nHere is a longer description of the function. It can be multiple lines long and\ncan contain any information that is relevant to the function.\n\nThe function returns: A list of person information, each having a name and city and optional other fields\n\nThe return type is: {'type': 'array', 'items': {'type': 'object', 'additionalProperties': {'type': 'string'}}}",
  'parameters': {'type': 'object',
   'properties': {'parm1': {'type': 'string',
     'description': 'the first parameter'},
    'parm2': {'type': 'object',
     'additionalProperties': {'type': 'array', 'items': {'type': 'integer'}},
     'description': 'the second parameter'},
    'parm3': {'type': 'string',
     'description': 'the third parameter',
     'default': True},
    'parm4': {'type': 'array!',
     'items': {'type': 'object',
      'properties': {'name': {'type': 'string'}, 'city': {'type': 'string'}}},
     'description': 'the fourth

In [11]:
llms.make_tooling(query_names)

[{'name': 'query_names',
  'description': 'Query the customer database and return a list of matching names. \n\nThis function queries the customer database using the conditions in the where clause and returns\na list of matching customers. The where clause may use the DB fields "city", "company_name",\n"country" and "since_date" to limit the returned customer list. The where clause can also \nbe followed by a limit clause to limit the number of returned names.\n\nThe function returns: a list of matching customer names\n\nThe return type is: {\'type\': \'array\', \'items\': {\'type\': \'string\'}}',
  'parameters': {'type': 'object',
   'properties': {'where_clause': {'type': 'string',
     'description': 'the string containing the where and optionally limit clauses in SQL query format'}},
   'required': ['where_clause']}}]